In [1]:
from pathlib import Path

import geopandas as gpd
from shapely.geometry import LineString, Point

from bressen import BasisGegevens, read_files
from bressen.geometries import (
    get_closest_feature,
    get_containing_feature,
    get_offsets,
    project_point,
)
from bressen.kering import get_kering_geometry
from bressen.styles import add_styles_to_geopackage

## Paden naar gegevens
Gegevens zijn aangeleverd in verschillende directories:
- 01_AGV_Dijktrajecten_Legger: dijktrajecten van waterschap Amstel Gooi en Vecht
- 02_ARK_Kering: kering van het Amsterdam-Rijnkanaal
- 03_Doorbraaklocaties: de locaties van de doorbraken
- 04_Watervlakken: alle beschikbare watervlakken

BASISGEGEVENS_GPKG en BRESSEN_GPKG verwijzen naar de basisgegevens (keringen, peil- en watervlakken) en het bressen-bestand

In [2]:
DATA_DIR = Path(r"d:\projecten\D2405.waternet.bressen\01.gegevens")
BASISGEGEVENS_GPKG = DATA_DIR / "basisgegevens.gpkg"
BRESSEN_GPKG = DATA_DIR / "bressen.gpkg"

## Inlezen
Inlezen van basisgegevens en bressen

In [3]:
# Inlezen basisgegevens vanuit een GeoPackage
basis_gegevens = BasisGegevens.from_gpkg(BASISGEGEVENS_GPKG)

# Inlezen bressen vanuit meerdere bestanden naar een GeoDataFrame
bressen_gdf = read_files(
    [
        DATA_DIR.joinpath("03_Doorbraaklocaties", "240122_breslocaties_gelijk_groter_1000.shp"),
        DATA_DIR.joinpath("03_Doorbraaklocaties", "240212_breslocaties_kleiner_1000_angle.shp"),
        DATA_DIR.joinpath("03_Doorbraaklocaties", "240212_breslocaties_Waterkeringen_Blaeu_ARK.shp"),
    ]
)

# Vullen van de bressenklasse met basisgegevens en de bressen GeoDataFrame
bressen = Bressen.from_gdf(bressen_gdf=bressen_gdf, basisgegevens=basisgegevens)

NameError: name 'Bressen' is not defined

## Verwerken bressen
We hanteren de volgende instellingen:

- `offset`: offset van de bres t.o.v. kering in meter
- `breedte`: breedte van de bres haaks op de kering in meter
- `lengte`: lengte van de bres langs de kering in meter


### alle bressen in 1X

In [ ]:
bressen.bereken_vlakken(offset=50, lengte=200, breedte=40)

### enkele bressen 
Wanneer je 1 of meerdere bressen wilt processen met andere instellingen, kun je individuele bressen opvragen en als dusdanig processen.

In [ ]:
bres = bressen.get_bres(fid=1329)
bres.bereken_vlak(offset=40, lengte=150, breedte=20, basisgegevens=bressen.basisgegevens)

## Wegschrijven resultaat

In de laatste stap schrijven we de resultaten weg naar een GeoPackage. Deze bevat drie lagen
- `bressen` met de originele punten samengevoegd in 1 laag (punt-feature)
- `bresvlakken` met de bresvlakken hierboven berekend (polygoon-feature)
- `fouten` met alle vlakken die niet kunnen worden berekend (geen geometrie)

In [ ]:
bressen.to_gpkg(DATA_DIR / "resultaat.gpkg")